# Clustering Problem

## load sample dataset

In [1]:
# load sample dataset
from pycaret.datasets import get_data
data = get_data('jewellery')

,Age,Income,SpendingScore,Savings
0,58,77769,0.791329,6559.829923
1,59,81799,0.791082,5417.661426
2,62,74751,0.702657,9258.992965
3,59,74373,0.765680,7346.334504
4,87,17760,0.348778,16869.507130


## Functional API

In [2]:
from pycaret.clustering import *
s = setup(data, normalize = True) # standardize the data (normalize = True)

,Description,Value
0,Session id,4819
1,Original data shape,"(505, 4)"
2,Transformed data shape,"(505, 4)"
3,Numeric features,4
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,Normalize,True
9,Normalize method,zscore


## Create Model

In [3]:
# functional API
kmeans = create_model('kmeans')
print(kmeans)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.7325,1122.4249,0.3947,0,0,0


KMeans(n_clusters=4, random_state=4819)


| 指標                    | 解釋                                   | 評分標準                  |
| --------------------- | ------------------------------------ | --------------------- |
| **Silhouette**        | 輪廓係數：衡量資料點與本群與其他群的相對距離（介於 -1 和 1 之間） | 越高越好，接近 **1** 表示分群效果佳 |
| **Calinski-Harabasz** | 分群之間的距離與群內距離的比值                      | 越高越好，代表群與群之間越分明       |
| **Davies-Bouldin**    | 衡量群與群之間的「相似度」                        | 越低越好，代表群與群之間差異越大      |
| **Homogeneity**       | 所有群內樣本屬於同一類別的程度（需有 label 才有效）        | 0 表示未定義或資料無標籤         |
| **Rand Index**        | 與真實分群相比的相似度（需有真實標籤）                  | 0 表示未定義               |
| **Completeness**      | 每個類別的樣本都在同一群內的程度（需 label）            | 0 表示未定義               |

---

KMeans(n_clusters=4, random_state=4819)：
這表示系統自動選定了 4 個群集，並固定隨機種子為 4819（確保結果可重現）。

## Analyze Model

In [4]:
# functional API
evaluate_model(kmeans)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### 📊 PyCaret Clustering 分群圖解總整理（前五項）

#### 1️⃣ 2D Cluster PCA Plot（主成分分析降維圖）

- 使用 PCA 將高維資料降為 2 維進行視覺化
- 每個點代表一筆資料，顏色代表所屬叢集（Cluster）
- 適合快速觀察叢集之間是否有明確區隔
- 缺點：只能保留部分資訊，可能有資訊損失

---

#### 2️⃣ 3D t-SNE Plot for Clusters（非線性降維視覺化）

- 使用 t-SNE 進行非線性降維至 3 維
- 保留「局部結構」，更能反映資料真實的群聚情形
- 各群呈現出球狀聚集，表示模型分群品質良好
- 適合觀察複雜資料的內在分佈情形

---

#### 3️⃣ Elbow Plot（手肘圖法）

- 用來判斷 KMeans 分群的最佳群數 \( k \)
- x 軸：群數 \( k \)，y 軸：失真分數（Distortion Score）
- 找出「手肘點」（彎曲處）作為最佳分群數
- 範例：圖中顯示最佳為 \( k = 5 \)，但目前模型為 \( k = 4 \)

---

#### 4️⃣ Silhouette Plot（輪廓係數圖）

- 用來衡量每筆資料對其分群的「適合程度」
- 輪廓係數範圍：-1 ~ 1，越接近 1 表示分得越好
- 顏色代表不同叢集，條形長度為每筆資料的輪廓分數
- 可觀察分群是否有重疊，或某些群內分散度過大

---

#### 5️⃣ Distance Plot（MDS 距離圖）

- 使用 MDS（多維尺度分析）計算各群中心的相對距離
- 圓形代表叢集中心與擴散範圍
- 群與群之間的距離越遠越好，重疊越少表示分得清楚
- 可以視覺化叢集間的相似程度與分離程度

## Assign Model

assign_model() 的功能是：

把「模型偵測出來的結果」附加回原始資料集，包含每筆資料的「是否為異常（Anomaly）」與「異常分數（Anomaly_Score）」，這樣你可以觀察哪幾筆被標記為異常。

In [5]:
# functional API
result = assign_model(kmeans)
result.head()

,Age,Income,SpendingScore,Savings,Cluster
0,58,77769,0.791329,6559.830078,Cluster 0
1,59,81799,0.791082,5417.661621,Cluster 0
2,62,74751,0.702657,9258.993164,Cluster 0
3,59,74373,0.765680,7346.334473,Cluster 0
4,87,17760,0.348778,16869.507812,Cluster 1


## Predictions

In [6]:
# functional API
predictions = predict_model(kmeans, data = data)
predictions.head()

,Age,Income,SpendingScore,Savings,Cluster
0,-0.042287,0.062733,1.103593,-1.072467,Cluster 0
1,-0.000821,0.174811,1.102641,-1.303473,Cluster 0
2,0.123577,-0.021200,0.761727,-0.526556,Cluster 0
3,-0.000821,-0.031712,1.004705,-0.913395,Cluster 0
4,1.160228,-1.606165,-0.602619,1.012686,Cluster 1


## Save the model

In [7]:
# functional API
save_model(kmeans, 'best_model_clustering')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Age', 'Income', 'SpendingScore',
                                              'Savings'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('trained_model', KMeans(n_clusters=4, random_state=4819))]),
 'best_model_clustering.pkl')

## To load the model back in the environment:

In [8]:
# functional API
loaded_model = load_model('best_model_clustering')
print(loaded_model)

Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=FastMemory(location=/var/folders/ty/th__2_pn1js9hzn2y7l830wr0000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Age', 'Income', 'SpendingScore',
                                             'Savings'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('trained_model', KMeans(n_clusters=4, random_state=4819))])
